In [1]:
#====================================================================================================
# by kobongsoo . 2022-04-13
# 기존 distilbert 모델에 출력 dimension=768을 128 로 줄이는 예제임 
#
# => semantic search시, cosin simility 유사도 측정시, 기존 embedding dimenision을 768로 저장해 두어야 하는데,
# 이때 저장공간이 db이거나 메모리이면, 데이터가 많은 경우 메모리 낭비가 심하므로, 이를 128처럼 줄이는 방법이 필요함
#
# => 여기서는 기존 distilbert를 가지고, mydistilbert 128차원으로 줄여 embedding 값을 출력하고,
# 이때 실제 cosin 유사도를 측정 테스트 해보는 예시임.
#
# => cosin 유사도 측정시, 768이나 128일때 별반 차이 없은 것으로 여겨짐. 
#====================================================================================================

import torch
import torch.nn as nn
import numpy as np
import os
import time

from transformers import logging
logging.set_verbosity_error()

from transformers import AutoModel, AutoTokenizer
#os.sys.path.append('..')
from myutils import GPU_info, seed_everything, mlogging, pytorch_cos_sim

device = GPU_info()
seed_everything(111)
logger=mlogging(loggername="test", logfilename="../log/test")


logfilepath:bwdataset_2022-04-13.log
logfilepath:qnadataset_2022-04-13.log
True
device: cuda:0
cuda index: 0
gpu 개수: 1
graphic name: NVIDIA A30
logfilepath:../log/test_2022-04-13.log


In [2]:
# distilbert에 dimension을 줄여서 last_hidden_state를 출력하는 class
class MyDistilBertModel(nn.Module):
    
    def __init__(self, 
                model_path: str,      # 기존 huggingface distilbert 모델 경로
                in_dim: int = 768,    # 입력 dimension(기본=768)
                out_dim: int = 768,   # 출력 dimension(기본=768)
                nlabel: int = 2,      # 혹시 classificaiton에 사용하는 경우 label 계수  
                drop_rate: int = 0.1):# dropout 비율(기본=0.1)
        
        
        super().__init__()  #nn.Module 호출시 반드시 호출해야함
        
        # bert 모델 정의 : huggingface에 distilbert pretrained 모델 불러옴
        self.model = AutoModel.from_pretrained(model_path)
        
        
        self.activefunc = nn.Tanh()            # activefunc 함수 정의 : activation function은 Tanh 이용
        self.drop = nn.Dropout(drop_rate)      # dropout 레이어 정의
        self.fc = nn.Linear(in_dim, out_dim)   # fullyconnect 레이어 정의 : 입력 dim, 출력 dim
        
        #self.classifier = nn.Linear(out_dim, nlabel)  # classification 레이어 정의
        
    def forward(self,
               input_ids,
               #token_type_ids = None,  #distilbert에는 token_type_ids가 없다.
               attention_mask 
               ):
        output = self.model(input_ids, attention_mask)
        h = output.last_hidden_state  #distilber는 맨마지막 hidden state만 리턴한다.
      
        out1 = self.drop(h)
        out2 = self.activefunc(self.fc(out1))
        
        #logits = self.classifier(self.drop(pooled_h))
        return out2
    

In [3]:
# 모델과 tokenizer 설정
model_path = '../model/distilbert/distilbert-0331-TS-nli-0.1-10'
tokenizer = AutoTokenizer.from_pretrained(model_path)

mymodel = MyDistilBertModel(model_path=model_path, in_dim=768, out_dim=128)
#mymodel = AutoModel.from_pretrained(model_path)

print(mymodel)

MyDistilBertModel(
  (model): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(167550, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Linear(in

In [4]:
# cosin 유사도 측정을 위한 문장들 정의

test_sentence = [
    '오늘은 비가 오고 날씨가 흐리겠다',
    '오늘은 눈이 오고 날씨가 흐리겠다',
    '오늘은 가끔 흐리고, 눈이 올수 있다',
    '여기 식당은 파스타가 맛있다',
    '오늘 증시는 내림으로 마감 하였다',
    '내일은 오전에는 흐리지만, 오후에는 날씨가 좋겠다',
    '서울은 대한민국에 수도이며, 정치 경제 중심지이다',
    '내년 경제 성장은 4%대 성장을 이룰거라 예상된다',
    '프랑스 파리는 전세계 관광객들이 매년 찾는 관광도시이다',
    '올해에는 대통령 선거와 지방선거가 동시에 열린다',
    '오늘 날씨는 비가 내리고 매우 춥다',
    '손홍민이 영국 프리미어 축구 경기에서 11번째 골을 넣었다',
    '건조한 날씨에 산불을 조심해야 한다',
    '윈도우11 OS에 검색 기능을 강화 하였다',
    '한국은행은 올해 하반기 금리를 동결했다',
    '날씨가 좋다',
    '안 그래도 되는데 뭐. 괜찮아.',
    '내일 저녁까지 보수 공사가 끝날 것으로 예상합니다.',
    '감사합니다. 후회 없는 결정이 될 겁니다.',
    '그러면 오전 10시에 보도록 하죠. 내일 뵙겠습니다.',
    '프린트 카트리지가 다 떨어졌습니다. 더 주문할까요?'
    ]

# ** 멀티로 한번에 tokenizer 할때는 반드시 padding=True 해야 함.(그래야 최대 길이 token에 맞춰서 padding 됨)
test = tokenizer(test_sentence, 
                 add_special_tokens=True, 
                 truncation=True, 
                 padding=True,   
                 max_length=256, 
                 return_tensors="pt")

print(test)

{'input_ids': tensor([[   101, 122278,  10892,   9379,  11287,   9580,  11664, 123665,  11287,
          10015,  12692, 118632,  11903,    102,      0,      0,      0,      0],
        [   101, 122278,  10892,   9034,  10739,   9580,  11664, 123665,  11287,
          10015,  12692, 118632,  11903,    102,      0,      0,      0,      0],
        [   101, 122278,  10892,   8843, 118707,  10015,  62211,    117,   9034,
          10739,   9583,  15891,  11506,    102,      0,      0,      0,      0],
        [   101, 119676, 123323,  10892, 136093,  11287,   9254,  76820,    102,
              0,      0,      0,      0,      0,      0,      0,      0,      0],
        [   101, 122278, 140417,  11018, 131258,  11467, 121653,  28750,    102,
              0,      0,      0,      0,      0,      0,      0,      0,      0],
        [   101, 129345,  10892, 120578,  15303,  10015, 154112,  19105,    117,
         120233,  15303, 123665,  11287,   9685, 118632,  11903,    102,      0],
        

In [5]:
# my모델에 출력값 추출
last_hidden_state = mymodel(**test)
#output = mymodel(**test)
#last_hidden_state = output.last_hidden_state

print(type(last_hidden_state)), print(last_hidden_state.shape)

<class 'torch.Tensor'>
torch.Size([21, 18, 128])


(None, None)

In [6]:
# 문장 유사도 비교

start = time.time()

# 첫번째 문장을 query로 지정함
in_mean_sequence = torch.mean(last_hidden_state[0], dim=0)
#print(in_mean_sequence.shape)

out_dict = {}
# for문을 돌면서 유사도 비교
for idx, hidden in enumerate(last_hidden_state):
    out_mean_sequence = torch.mean(hidden, dim=0)
    simul_score = pytorch_cos_sim(in_mean_sequence, out_mean_sequence)
    #print("input vs {:d} 유사도:{}".format(idx, simul_score))
    
     # 사전 key로 순번으로 하고, 유사도를 저장함
    key = str(idx+1)
    out_dict[key] = simul_score
    #print("input vs {:d} 유사도:{}".format(idx, simul_score))
    
#print(out_dict)

# 사전 정렬(value(유사도)로 reverse=True 하여 내림차순으로 정렬함)
sorted_dict = sorted(out_dict.items(), key = lambda item: item[1], reverse = True)
#print(sorted_dict)

# 내립차순으로 정렬된 사전출력 
logger.info(f'---------------------------------------------------------')
for count in (sorted_dict):
    value = count[1].tolist() # count[1]은 2차원 tensor이므로 이를 list로 변환
    index = int(count[0])
    #print(test_sentence[index-1])
    logger.info(f'{test_sentence[index-1]}, 유사도:{value[0][0]}')

logger.info(f'---------------------------------------------------------')
logger.info(f'=== 유사도 처리시간: {time.time() - start:.3f} 초 ===')
logger.info(f'-END-\n')

2022-04-13 10:38:47,654 - test - INFO - ---------------------------------------------------------
2022-04-13 10:38:47,656 - test - INFO - 오늘은 비가 오고 날씨가 흐리겠다, 유사도:1.000000238418579
2022-04-13 10:38:47,657 - test - INFO - 오늘은 눈이 오고 날씨가 흐리겠다, 유사도:0.9396036863327026
2022-04-13 10:38:47,657 - test - INFO - 오늘 날씨는 비가 내리고 매우 춥다, 유사도:0.8798013925552368
2022-04-13 10:38:47,658 - test - INFO - 안 그래도 되는데 뭐. 괜찮아., 유사도:0.8244172930717468
2022-04-13 10:38:47,659 - test - INFO - 그러면 오전 10시에 보도록 하죠. 내일 뵙겠습니다., 유사도:0.7822104096412659
2022-04-13 10:38:47,660 - test - INFO - 감사합니다. 후회 없는 결정이 될 겁니다., 유사도:0.7809420824050903
2022-04-13 10:38:47,661 - test - INFO - 내일 저녁까지 보수 공사가 끝날 것으로 예상합니다., 유사도:0.683722734451294
2022-04-13 10:38:47,661 - test - INFO - 내일은 오전에는 흐리지만, 오후에는 날씨가 좋겠다, 유사도:0.643785834312439
2022-04-13 10:38:47,662 - test - INFO - 건조한 날씨에 산불을 조심해야 한다, 유사도:0.6342536807060242
2022-04-13 10:38:47,663 - test - INFO - 한국은행은 올해 하반기 금리를 동결했다, 유사도:0.588422417640686
2022-04-13 10:38:47,664 - test - INFO

In [11]:
#mydistilbertmodel 저장
# => mydistilbertmodel은 새로 정의한 model이므로, save_pretrained 함수 이용 못함.따라서 torch.save로 모델 저정해야 함
OUTPATH = '../model/distilbert/mydistilbert/'
os.makedirs(OUTPATH, exist_ok=True)
torch.save(mymodel, OUTPATH + 'pytorch_model.bin') 
#mymodel.save_pretrained(OUTPATH)  # save_pretrained 로 저장하면 config.json, pytorch_model.bin 2개의 파일이 생성됨
tokenizer.save_pretrained(OUTPATH)

('../model/distilbert/mydistilbert/tokenizer_config.json',
 '../model/distilbert/mydistilbert/special_tokens_map.json',
 '../model/distilbert/mydistilbert/vocab.txt',
 '../model/distilbert/mydistilbert/added_tokens.json',
 '../model/distilbert/mydistilbert/tokenizer.json')

In [16]:
# 불러올때는 torch.load로 불러옴
model = torch.load(OUTPATH+'pytorch_model.bin')
print(model)

MyDistilBertModel(
  (model): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(167550, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Linear(in